In [36]:
# please put all the package used here
from collections import Counter
import pandas as pd
import csv
import pprint
import numpy as np
import warnings
import requests
import json
warnings.filterwarnings('ignore')
import psycopg2
import psycopg2.extras
import time
import ast # convert string to dictionary
from scipy import stats
from scipy.stats import zscore

In [41]:
data_dic = list(csv.DictReader(open('finals.csv')))
data = pd.read_csv('finals.csv')

data_BusinessStats = pd.read_csv('new_BusinessStats.csv')

In [9]:
data.head()

,area_id,area_name,land_area,population,number_of_dwellings,carpods_num,population_density,dwelling_density,transport_density,ZP,ZD,ZT
0,102011028,Avoca Beach - Copacabana,643.8,7590.0,2325,0,11.789376,3.611370,0.0,-1.127443,-0.953681,-0.740649
1,102011029,Box Head - MacMasters Beach,3208.6,10986.0,3847,0,3.423923,1.198965,0.0,-0.695131,-0.271481,-0.740649
2,102011030,Calga - Kulnura,76795.1,4841.0,1575,0,0.063038,0.020509,0.0,-1.477391,-1.289851,-0.740649
3,102011031,Erina - Green Point,3379.3,14237.0,4450,0,4.213003,1.316841,0.0,-0.281277,-0.001201,-0.740649
4,102011032,Gosford - Springfield,1691.2,19385.0,6373,0,11.462275,3.768330,0.0,0.374065,0.860737,-0.740649


In [10]:
len(data)

312

In [11]:
for i in data['carpods_num']:
    if i > 0:
        #print(i) 

17
1
40
15
10
9
13
63
1
2
7
2
18
77
3
9
5
2
2
2
21
21
12
14
6
2
5
8
1
2
3
1
3
13
8
3
22


In [13]:
len(data["area_name"].unique())

312

# Balance 

In [88]:
data_BusinessStats = pd.read_csv('new_BusinessStats1.csv')
data_generate= data_BusinessStats
data_BusinessStats.head()

,area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services
0,102011028,738.0,35.0,33.0,62.0,9.0,20.0
1,102011029,907.0,42.0,21.0,43.0,11.0,29.0
2,102011030,1102.0,42.0,17.0,11.0,18.0,23.0
3,102011031,1666.0,143.0,68.0,164.0,22.0,15.0
4,102011032,2126.0,159.0,95.0,230.0,36.0,27.0


In [89]:
#data_BusinessStats = pd.read_csv('new_BusinessStats.csv')
data_generate= data_BusinessStats
data_BusinessStats.head()

,area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services
0,102011028,738.0,35.0,33.0,62.0,9.0,20.0
1,102011029,907.0,42.0,21.0,43.0,11.0,29.0
2,102011030,1102.0,42.0,17.0,11.0,18.0,23.0
3,102011031,1666.0,143.0,68.0,164.0,22.0,15.0
4,102011032,2126.0,159.0,95.0,230.0,36.0,27.0


In [90]:
data_generate['retail_trade_z'] = data_generate['retail_trade']/data_generate['num_businesses']
data_generate['accommodation_and_food_services_z'] = data_generate['accommodation_and_food_services']/data_generate['num_businesses']
data_generate['health_care_and_social_assistance_z'] = data_generate['health_care_and_social_assistance']/data_generate['num_businesses']
data_generate['education_and_training_z'] = data_generate['education_and_training']/data_generate['num_businesses']
data_generate['arts_and_recreation_services_z'] = data_generate['arts_and_recreation_services']/data_generate['num_businesses']
data_generate.head()

,area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services,retail_trade_z,accommodation_and_food_services_z,health_care_and_social_assistance_z,education_and_training_z,arts_and_recreation_services_z
0,102011028,738.0,35.0,33.0,62.0,9.0,20.0,0.047425,0.044715,0.084011,0.012195,0.027100
1,102011029,907.0,42.0,21.0,43.0,11.0,29.0,0.046307,0.023153,0.047409,0.012128,0.031974
2,102011030,1102.0,42.0,17.0,11.0,18.0,23.0,0.038113,0.015426,0.009982,0.016334,0.020871
3,102011031,1666.0,143.0,68.0,164.0,22.0,15.0,0.085834,0.040816,0.098439,0.013205,0.009004
4,102011032,2126.0,159.0,95.0,230.0,36.0,27.0,0.074788,0.044685,0.108184,0.016933,0.012700


In [91]:
df=data_generate
cols = list(df.columns)
cols.remove('retail_trade')
cols.remove('accommodation_and_food_services')
cols.remove('health_care_and_social_assistance')
cols.remove('education_and_training')
cols.remove('arts_and_recreation_services')
cols.remove('area_id')
cols.remove('num_businesses')
df[cols].head()

,retail_trade_z,accommodation_and_food_services_z,health_care_and_social_assistance_z,education_and_training_z,arts_and_recreation_services_z
0,0.047425,0.044715,0.084011,0.012195,0.027100
1,0.046307,0.023153,0.047409,0.012128,0.031974
2,0.038113,0.015426,0.009982,0.016334,0.020871
3,0.085834,0.040816,0.098439,0.013205,0.009004
4,0.074788,0.044685,0.108184,0.016933,0.012700


In [92]:
def z_score(df):
    df.columns = [x + "_zscore" for x in df.columns.tolist()]
    return ((df - df.mean())/df.std(ddof=0))

In [93]:
df[cols]=z_score(df[cols])

In [94]:
df.tail()

,area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services,retail_trade_z,accommodation_and_food_services_z,health_care_and_social_assistance_z,education_and_training_z,arts_and_recreation_services_z
307,128021537,3.0,0.0,3.0,0.0,0.0,0.0,-3.361457,16.835211,-1.948574,-2.299213,-1.404769
308,128021538,2086.0,107.0,79.0,134.0,36.0,27.0,-0.373485,-0.034120,0.189632,0.396357,-0.036263
309,128021607,1121.0,75.0,38.0,76.0,28.0,11.0,0.535833,-0.103784,0.308090,1.602137,-0.367278
310,128021608,417.0,17.0,6.0,17.0,7.0,7.0,-0.986696,-0.445856,-0.591599,0.322737,0.370071
311,128021609,259.0,10.0,5.0,10.0,0.0,7.0,-1.112365,-0.359653,-0.663408,-2.299213,1.452792


In [95]:
df.to_csv("new_BusinessStats.csv",index=False)

In [ ]:
balance = {} 
for i in df["area_id"]:
        if i not in data['area_id'].unique():
            print("error {}".format(i) ) 
        else:
            balance[i] += 
            

## Walkability score

In [ ]:
walkability = {} 
count=0
for i in data["area_id"]:
     score = 
        else:
            if i in user_totalNr_inv.keys():
                walkability[i] += 1
            else:
                walkability[i] = 1
        count += 1